# DNBLab Workshop: Daten bereinigen und zusammenführen

# Part 1: Datenbezug mittels SRU-Schnittstelle

Als Datenbasis dient das Digitalisierungsprojekt "100 Bände Klassik". Es enthält namenhafte klassische Werke u.a. von Theodor Fontane, J.W. von Goethe und Rainer Maria Rilke und eignet sich daher besonders für einen ersten Einstieg in die Datenanreicherung, da die AutorInnen bereits umfassende Einträge in der GND haben. 

Die Daten werden mittels SRU-Schnittstelle bezogen und zur weiteren Verarbeitung in einer .csv Datei gespeichert. 

## Einrichten der Arbeitsumgebung

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird Requests https://docs.python-requests.org/en/latest/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden.

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd
import matplotlib.pyplot as plt

## SRU-Abfrage mit Ausgabe in MARC21-xml

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von Records aus. Bei mehr als 100 Records werden weitere Datensätze mit "&startRecord=101" abgerufen (mögliche Werte 1 bis 99.000). Weitere Informationen und Funktionen der SRU- Schnittstelle werden unter https://www.dnb.de/sru beschrieben.

In [2]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

# Durchsuchen eines MARC-Feldes

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen heraus und liefert diese als Dictionary zurück. Die Schlüssel-Werte-Paare können beliebig agepasst und erweitert werden. In diesem Fall werden Elemente für IDN und Titel geliefert.

Tipp! Die SRU Abfrage https://services.dnb.de/sru/dnb?version=1.1&operation=searchRetrieve&query=cod=d002&recordSchema=MARC21-xml&maximumRecords=100 kann dazu genutzt werden, um die Marc-Tags und Unterfelder für bestimmte, gesuchte Inhalte zu finden. 

In [3]:
def parse_record(record):
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    # IDN
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # Titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unknown"
    
    # Erscheinungsjahr
    jahr = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    try:
        jahr = jahr[0].text
    except:
        jahr = "unknown"
    
    # Verfasserangabe
    verfasser = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        verfasser = verfasser[0].text
    except:
        verfasser = "unknown"
    
    # GND-ID
    gnd_id = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = '0']", namespaces=ns)
    try:
        gnd_id = gnd_id[0].text
    except:
        gnd_id = "unknown"
    
    # URN
    urn = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    try:
        urn = urn[0].text
    except:
        urn = "unknown"
    
    # Verlag
    verlag = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'b']", namespaces=ns)
    try:
        verlag = verlag[0].text
    except:
        verlag = "unknown"
    
    # Verlagsort
    verlagsort = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        verlagsort = verlagsort[0].text
    except:
        verlagsort = "unknown"
        
    meta_dict = {
        "idn": idn,
        "titel": titel,
        "jahr": jahr,
        "verfasser": verfasser,
        "gnd_id": gnd_id,
        "urn": urn,
        "verlag": verlag,
        "verlagsort": verlagsort
    }
    
    return meta_dict


In [4]:
records = dnb_sru('cod=d002')
print(len(records), 'Ergebnisse')

C:\Users\fuchsf\AppData\Local\Temp\ipykernel_10184\1107150534.py:11: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  xml = soup(r.content)


108 Ergebnisse


C:\Users\fuchsf\AppData\Local\Temp\ipykernel_10184\1107150534.py:26: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  xml = soup(r.content)


## CSV Download

Für die Datenbereinigung und Datenanreicherung wird die Arbeit im .csv Format empfohlen, weswegen die Suchergebnisse im folgenden in einem Dataframe ausgegeben und anschließend für die weitere Bearbeitung heruntergeladen werden. 

In [5]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,titel,jahr,verfasser,gnd_id,urn,verlag,verlagsort
0,1003104487,Egmont,[1946],"Goethe, Johann Wolfgang von",(DE-588)118540238,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Schöningh,Paderborn
1,999490184,Das Amulett,[1939],"Meyer, Conrad Ferdinand",(DE-588)118581775,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Verl. Dt. Volksbücher,Wiesbaden
2,1000047377,Der Struwwelpeter oder lustige Geschichten u...,[1939],"Hoffmann, Heinrich",(DE-588)11855249X,https://nbn-resolving.org/urn:nbn:de:101:2-201...,[Loewe],[Stuttgart]
3,1000290328,Der Zweikampf,1939,"Kleist, Heinrich von",(DE-588)118563076,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Kohlhammer,Stuttgart
4,99962461X,Heidi,1939,"Spyri, Johanna",(DE-588)118616455,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Rascher,Zürich
...,...,...,...,...,...,...,...,...
103,1000746348,Leyer und Schwerdt,1913,"Körner, Theodor",(DE-588)118713507,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Morawe & Scheffelt,Berlin
104,100003917X,Schillers Wallenstein,[1913],"Schiller, Friedrich",(DE-588)118607626,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Dt. Bibliothek,Berlin
105,1000062104,Vor dem Sturm,1913,"Fontane, Theodor",(DE-588)118534262,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Cotta,Stuttgart
106,1000775615,Der Tod des Tizian,[1912],"Hofmannsthal, Hugo von",(DE-588)118552759,https://nbn-resolving.org/urn:nbn:de:101:2-201...,Insel-Verl.,Leipzig


In [6]:
# DataFrame als CSV speichern
df.to_csv('Klassik.csv', index=False)